# Getting Even - Coding Challenge - F#

## Description

A coding challenge:
1. There is a 4x6 grid.
2. 18 spots must be filled in.
3. The number of spots in each row &amp; column must be even.
4. Print out a 4x6 grid (or equivalent) showing the first solution that you find.

### Stretch Goal #1
* Print out all possible solutions.

### Stretch Goal #2
* Print out only the symmetric solutions, i.e. solutions which are unchanged if you reverse the rows, the columns, or both.

## Solution Discussion
The aim here is to create a functional solution in F#, for an arbitrary MxN grid.

As an additional "trick", optimise for the case where the number of spots is > (M+N)/2 by instead solving for "MxN - number of spots" and then inverting the result.

## Setup

In [73]:
let Mrows: int = 4 // must be even and > 0
let Ncols: int = 6 // must be even and > 0
let NumSpots: int = 18 // must be even and >= 4 and < Mrows*Ncols

In [74]:
let isEven x = (x % 2) = 0

let boolToInt b =
    match b with
    | true -> 1
    | false -> 0

let invert (b:Boolean) = not b

let check (testValue: Boolean) (checkMessage: String) =
    if (not testValue) then raise (Exception checkMessage)

let checkGridParams (mrows:int) (ncols:int): unit =
    check ((mrows > 0) && isEven(mrows)) (sprintf "invalid mrows: %d" mrows)
    check ((ncols > 0) && isEven(ncols)) (sprintf "invalid ncols: %d" ncols)

let checkProblemParams (mrows:int) (ncols:int) (numSpots:int): unit =
    checkGridParams mrows ncols
    let maxSpots = mrows*ncols
    check ((numSpots >= 4) && (numSpots < maxSpots) && isEven(numSpots)) (sprintf "invalid numSpots: %d" numSpots)

In [75]:
let MaxSpots = Mrows * Ncols
let OptimalNumSpots = Math.Min(NumSpots, MaxSpots - NumSpots)
printfn "#spots = %d; optimal #spots = %d" NumSpots OptimalNumSpots

#spots = 18; optimal #spots = 6


## Grid Implementation

We need to support the concept of a grid of Boolean "spots" that we can set (to true) or reset (to false).

In [76]:
let updateList<'a> (newValue:'a) (index:int) (aList: 'a list): 'a list =
    List.concat [aList |> List.take (index); [newValue] ; aList |> List.skip (index+1)]

let rec ListToReversedCols<'a> (rows: int) (acc: 'a list list) (isSpot: 'a list): 'a list list =
    match isSpot with
    | [] -> acc
    | spotList ->
        let nextCol = isSpot |> List.take rows
        ListToReversedCols rows (nextCol::acc) (isSpot |> List.skip rows)

let SpotsToString (isSpot: Boolean list): String =
    isSpot |> List.map (fun spot -> if (spot) then "X" else "_") |> String.concat " "

Note that ```Grid``` is a Discriminated Union (record type) with members (operations).  Does that mean this is object-oriented code?  Not in this case - it's just a design choice on my part, I could equally have created a module where each member would just be a function in the module, as you would do in many functional programming languages.

In [77]:
type Grid =
    { Rows: int; Cols: int; IsSpot: Boolean list } // IsSpot is column-by-column list of Boolean values

    member this.CheckValid(): unit =
        assert(this.IsSpot.Length = this.Rows*this.Cols)
    
    member this.GetColumns(): Boolean list list =
        let reversedResult: Boolean list list = ListToReversedCols this.Rows [[]] this.IsSpot
        reversedResult |> List.rev |> List.tail // reverse the results list and remove the initial empty list

    member this.GetRows(): Boolean list list =
        let cols = this.GetColumns()
        seq{
            for idx in 0 .. (this.Rows-1) do
                yield cols |> List.map (fun col -> col.[idx])
         } |> Seq.toList
    
    member this.ToGridString(): String =
        let rows = this.GetRows()
        rows |> List.map SpotsToString |> String.concat "\n"

    member this.UpdateIndex (index: int) (newValue:Boolean): Grid =
        let result = {this with Grid.IsSpot = (updateList newValue index this.IsSpot)}
        result.CheckValid()
        result
    
    member this.Update (row:int) (col:int) (newValue:Boolean): Grid =
        this.UpdateIndex (col*this.Rows+row) newValue
            
    member this.Set (row:int) (col:int): Grid = this.Update row col true

    member this.Reset (row:int) (col:int): Grid = this.Update row col false

    member this.SetIndices (indices: int list): Grid =
        match indices with
        | [] -> this
        | head::tail -> (this.UpdateIndex head true).SetIndices(tail)
    
    member this.ReverseColumns(): Grid =
        let reversedColumns = this.GetColumns() |> List.rev
        { this with Grid.IsSpot = (reversedColumns |> List.concat) }

    member this.ReverseRows(): Grid =
        let reversedColumns = this.GetColumns() |> List.map (List.rev)
        { this with Grid.IsSpot = (reversedColumns |> List.concat) }
    
    member this.IsValidSolution(): Boolean = // check if every row and column has an even number of "spots"
        let columnsValid = (this.GetColumns() |> List.map (fun column -> column |> List.map boolToInt |> List.sum |> isEven) |> List.filter (fun bool -> not bool) |> List.length) = 0
        if (columnsValid)
        then
            let rowsValid = (this.GetRows() |> List.map (fun column -> column |> List.map boolToInt |> List.sum |> isEven) |> List.filter (fun bool -> not bool) |> List.length) = 0
            rowsValid
        else false
    
    member this.Invert: Grid = {this with Grid.IsSpot = this.IsSpot |> List.map invert}

    static member NewFalseGrid (mrows:int) (ncols:int): Grid =
        let result = {Rows = mrows; Cols = ncols; IsSpot = (List.init (mrows*ncols) (fun _ -> false))}
        result.CheckValid()
        result

Let's just test some of this code.

In [78]:
updateList false 1 [true;true;true]

[ True, False, True ]

In [79]:
let grid = (Grid.NewFalseGrid 3 5).SetIndices([0;2;6;10;14])
grid.CheckValid()
grid.ToGridString()

X _ X _ _
_ _ _ X _
X _ _ _ X

In [80]:
grid.ReverseColumns().ToGridString()

_ _ X _ X
_ X _ _ _
X _ _ _ X

In [81]:
grid.ReverseRows().ToGridString()

X _ _ _ X
_ _ _ X _
X _ X _ _

In [82]:
(grid.Set 0 1).ToGridString()

X X X _ _
_ _ _ X _
X _ _ _ X

## Solution Logic

```initRange``` returns an increasing list of values, of the given length, starting with the given value.

In [83]:
let initRange (startIndex: int) (length: int): (int list) =
    seq { for idx in (startIndex)..(startIndex+length-1) do yield idx } |> Seq.toList

In [84]:
initRange 3 4

[ 3, 4, 5, 6 ]

```nextIndices``` steps from the current indices to the next possible set of indices.  If there is no next possible set of indices, it returns 'None'.

Importantly, each index is an index to a location in a grid, so:
* We don't want any two indices to be the same, because we can't have two "spots" at the same point in the grid.
* We can also demand that the indices be either strictly increasing or strictly decreasing,
  which avoids visiting multiple permutations of the same set of indices.
  We don't want to visit any more than a single permutation of a set of indices.

Here, we start each index from zero, the minimum, to produce the next valid set of strictly increasing indices, increasing the right-most index that can be updated.

_Note: ```nextIndices``` is the most important part of the solution, and it is the part that took me to longest to implement correctly, after many rounds of trial and error._

In [85]:

let rec nextIndices (minIndex: int) (maxIndex: int) (indices: int list): (int list) option =
    //printfn "indices: %A" indices
    match indices with
    | [] -> None
    | _ ->
        let result =
            match indices.Head with
            | idx when idx = maxIndex -> // head index is already as large as allowed
                //printfn "#1: None"
                None
            | idx -> // can increment the head of indices, but try to increment the tail first
                if (indices.Tail.Length >= 1) // if the tail is non-empty, try to increment the tail first
                then
                    let newTail = nextIndices (indices.Head+1) maxIndex indices.Tail
                    match newTail with
                    | None -> // no possible new tail, try incrementing the head instead
                        let newHead = indices.Head + 1
                        let initTail = initRange (newHead+1) indices.Tail.Length
                        if (initTail |> List.max) <= maxIndex
                        then
                            //printfn "#2: %A" (Some(newHead::initTail))
                            Some(newHead::initTail)
                        else
                            //printfn "#3: None"
                            None
                    | Some(tail) ->
                        //printfn "#4: %A" (Some(idx::tail))
                        Some(idx::tail)
                elif (idx < maxIndex) // otherwise increase the sole index is possible
                then
                    //printfn "#5: %A" (Some([idx+1]))
                    Some([idx+1])
                else // otherwise return None
                    //printfn "#6: None"
                    None
        //printfn "#7: %A" result
        result

In [86]:
initRange 0 4

[ 0, 1, 2, 3 ]

In [87]:
nextIndices 0 4 (initRange 0 4)

Value,"[ 0, 1, 2, 4 ]"


In [88]:
nextIndices 0 4 [0;1;2;4]

Value,"[ 0, 1, 3, 4 ]"


In [89]:
nextIndices 0 4 [0;1;3;4]

Value,"[ 0, 2, 3, 4 ]"


In [90]:
nextIndices 0 4 [0;2;3;4]

Value,"[ 1, 2, 3, 4 ]"


In [91]:
nextIndices 0 4 [1;2;3;4]

<null>

Now we look for a solution iteratively, by modifying the index locations of the "spots" and looking for the first spot configuration that is a solution.

In [92]:
let iterateSolutionPrintPeriod = 1000

let solveGrid (mrows:int) (ncols:int) (numSpots:int): Grid list =
    let mutable iterationCount = 0
    checkProblemParams mrows ncols numSpots
    let maxIndex = mrows*ncols - 1
    let mutable loopIndices = Some(initRange 0 numSpots)
    let allIndexListsSeq = seq{
        while (loopIndices.IsSome) do
            yield loopIndices.Value
            loopIndices <- nextIndices 0 maxIndex loopIndices.Value
    }
    let allIndexLists = allIndexListsSeq |> Seq.toList
    let startGrid = Grid.NewFalseGrid mrows ncols
    let allGrids = allIndexLists |> List.map (fun indices -> startGrid.SetIndices indices)
    let solutionGrids = allGrids |> List.filter (fun grid -> grid.IsValidSolution())
    solutionGrids

## Run Solution

In [93]:
let grids = solveGrid Mrows Ncols OptimalNumSpots
grids.Length

480

In [94]:
if (grids.Length >= 1)
then
    printfn "#solutions: %d\n" (grids.Length)
    if (OptimalNumSpots = NumSpots)
    then
        for grid in grids do
            printfn "%s" (grid.ToGridString())
            printfn ""
    else
        for grid in grids do
            printfn "%s" (grid.Invert.ToGridString())
            printfn ""
else
    printfn "FAILED: no solution found"

#solutions: 480

_ _ X X X X
_ X _ X X X
X _ _ X X X
X X X X X X

_ _ X X X X
_ X X _ X X
X _ X _ X X
X X X X X X

_ _ X X X X
_ X X X _ X
X _ X X _ X
X X X X X X

_ _ X X X X
_ X X X X _
X _ X X X _
X X X X X X

_ _ X X X X
_ X _ X X X
X X X X X X
X _ _ X X X

_ _ X X X X
_ X X _ X X
X X X X X X
X _ X _ X X

_ _ X X X X
_ X X X _ X
X X X X X X
X _ X X _ X

_ _ X X X X
_ X X X X _
X X X X X X
X _ X X X _

_ X _ X X X
_ _ X X X X
X _ _ X X X
X X X X X X

_ X X _ X X
_ _ X X X X
X _ X _ X X
X X X X X X

_ X X X _ X
_ _ X X X X
X _ X X _ X
X X X X X X

_ X X X X _
_ _ X X X X
X _ X X X _
X X X X X X

_ X _ X X X
_ _ X X X X
X X X X X X
X _ _ X X X

_ X X _ X X
_ _ X X X X
X X X X X X
X _ X _ X X

_ X X X _ X
_ _ X X X X
X X X X X X
X _ X X _ X

_ X X X X _
_ _ X X X X
X X X X X X
X _ X X X _

_ X _ X X X
_ X X _ X X
X X _ _ X X
X X X X X X

_ X _ X X X
_ X X X _ X
X X _ X _ X
X X X X X X

_ X _ X X X
_ X X X X _
X X _ X X _
X X X X X X

_ X _ X X X
_ X X _ X X
X X X X X X
X X _ _ X X

_ X

In [95]:
let symmetricGrids = grids |> List.filter (fun g -> (g = g.ReverseColumns()) && (g = g.ReverseRows()))
symmetricGrids.Length

0

In [96]:
if (symmetricGrids.Length >= 1)
then
    printfn "# symmetric solutions: %d\n" (symmetricGrids.Length)
    if (OptimalNumSpots = NumSpots)
    then
        for grid in symmetricGrids do
            printfn "%s" (grid.ToGridString())
            printfn ""
    else
        for grid in symmetricGrids do
            printfn "%s" (grid.Invert.ToGridString())
            printfn ""
else
    printfn "FAILED: no symmetric solution found"

FAILED: no symmetric solution found


## Results

### 4x4 with 4 spots
* 36 solutions
* 4 symmetric solutions

### 4x4 with 12 spots
* 36 solutions
* 4 symmetric solutions

### 4x6 with 4 spots
* 90 solutions
* 6 symmetric solutions

### 4x6 with 18 spots
* 480 solutions
* 0 symmetric solutions (yes, really zero).

## Discussion
With a 4x6 grid and 18 spots, equivalent to a 4x6 grid and 6 holes, with an even number of spots in every row and column, meaning also an even number of holes in every row and column, it must be the case that:
* Three rows in each valid solution have two holes each, i.e. 4 spots each, while one has no holes, i.e. 6 spots.
* Three columns in each valid solution have two holes each, i.e. 2 spots each, while three columns have no holes, i.e. 4 spots.

This could also be used as a way to generate possible solutions.

**Note that it also means that there aren't any symmetric solutions - as one row differs from the other three rows.**